In [18]:
import struct
import numpy as np
import array
import scipy.sparse
import scipy.optimize
from softmax import SoftmaxRegression

In [19]:
def loadMNISTImages(file_name):

    image_file = open(file_name, 'rb')

    head1 = image_file.read(4)
    head2 = image_file.read(4)
    head3 = image_file.read(4)
    head4 = image_file.read(4)

    num_examples = struct.unpack('>I', head2)[0]
    num_rows = struct.unpack('>I', head3)[0]
    num_cols = struct.unpack('>I', head4)[0]

    dataset = np.zeros((num_rows * num_cols, num_examples))

    images_raw = array.array('B', image_file.read())
    image_file.close()

    for i in range(num_examples):
        limit1 = num_rows * num_cols * i
        limit2 = num_rows * num_cols * (i + 1)

        dataset[:, i] = images_raw[limit1: limit2]

    return dataset / 255

def loadMNISTLabels(file_name):

    label_file = open(file_name, 'rb')

    head1 = label_file.read(4)
    head2 = label_file.read(4)

    num_examples = struct.unpack('>I', head2)[0]

    labels = np.zeros((num_examples, 1), dtype=int)

    labels_raw = array.array('b', label_file.read())
    label_file.close()

    labels[:, 0] = labels_raw[:]

    return labels

def train(regressor):
    max_iterations = 100

    training_data = loadMNISTImages('./data/train-images.idx3-ubyte')
    training_labels = loadMNISTLabels('./data/train-labels.idx1-ubyte')

    opt_solution = scipy.optimize.minimize(regressor.softmaxCost, np.squeeze(regressor.theta),
                                       args=(training_data, training_labels,), method='L-BFGS-B',
                                       jac=True, options={'maxiter': max_iterations})
    
    regressor.theta = opt_solution.x
    return opt_solution.x
    

In [20]:
regressor = SoftmaxRegression()
opt_theta = train(regressor)

In [21]:
opt_theta.shape

(7840,)

In [22]:
test_data = loadMNISTImages('./data/t10k-images.idx3-ubyte')
test_labels = loadMNISTLabels('./data/t10k-labels.idx1-ubyte')

predictions = regressor.softmaxPredict(test_data)

correct = test_labels[:, 0] == predictions[:, 0]
print(f'Accuracy: {np.mean(correct)}')

Accuracy: 0.9055


In [23]:
test_data.shape

(784, 10000)

In [24]:
regressor.save_weights("./weight/softmax_weights.npz")